# Linear regression project - predicting the sale price of homes

# Data cleaning

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# read in training data
housing_train = pd.read_csv('../data/raw/train.csv', index_col='Id')

In [3]:
# read in test data
housing_test = pd.read_csv('../data/raw/test.csv', index_col='Id')

## 1. Data structure

In [4]:
housing_train.shape

(2051, 80)

In [5]:
housing_train.columns

Index(['PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config',
       'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish'

In [6]:
## for column definitions see: https://www.kaggle.com/c/dsi-us-4-project-2-regression-challenge/data 

In [7]:
housing_test.shape

(879, 79)

In [8]:
# check train and test sets have the same features
train_columns = set(housing_train.columns)
test_columns = set(housing_test.columns)

print('Features in training set but not in test set:', train_columns.difference(test_columns))
print('Features in test set but not in training set:', test_columns.difference(train_columns))

Features in training set but not in test set: {'SalePrice'}
Features in test set but not in training set: set()


In [9]:
## the features of both train and test sets are the same
## but the training set also has the target variable, as expected
## there are 79 features altogether

## 2. Data completeness

In [10]:
# look at columns with missing data in combined train and test set
housing_full = pd.concat([housing_train.drop(columns='SalePrice'), housing_test], axis=0)
housing_full['train'] = [idx in list(housing_train.index) for idx in housing_full.index] # label of train/text data

incomplete_columns = housing_full.isnull().sum()[housing_full.isnull().sum() !=0]
100*(incomplete_columns/housing_full.shape[0]).sort_values(ascending=False)

Pool QC           99.556314
Misc Feature      96.382253
Alley             93.242321
Fence             80.477816
Fireplace Qu      48.532423
Lot Frontage      16.723549
Garage Yr Blt      5.426621
Garage Cond        5.426621
Garage Qual        5.426621
Garage Finish      5.426621
Garage Type        5.358362
Bsmt Exposure      2.832765
BsmtFin Type 2     2.764505
Bsmt Cond          2.730375
Bsmt Qual          2.730375
BsmtFin Type 1     2.730375
Mas Vnr Type       0.784983
Mas Vnr Area       0.784983
Bsmt Half Bath     0.068259
Bsmt Full Bath     0.068259
Garage Cars        0.034130
BsmtFin SF 2       0.034130
BsmtFin SF 1       0.034130
Bsmt Unf SF        0.034130
Total Bsmt SF      0.034130
Garage Area        0.034130
Electrical         0.034130
dtype: float64

In [11]:
# check percentage of rows with missing data
1 - housing_full.dropna().shape[0]/housing_full.shape[0]

1.0

### Columns with legitimate 'NA' or 'None' category

In [12]:
## many categorical features have a legitimate 'NA' or 'None' category which may be registering as a 'nan' entry

In [13]:
# check categories with legitimate 'NA' values (see columns definitions) with cross-referencing

#### Useful functions

In [14]:
def get_inconsistent_rows(df_full, feature_subset):
    
    ''' returns rows of dataframe where there are inconsistent nan/zero values for a set a related features'''
    
    # get all rows with missing data
    incomplete_feature_index = df_full[feature_subset].isnull().any(axis=1)
    incomplete_feature_data = df_full.loc[incomplete_feature_index, feature_subset]

    # self-consistent rows are where all values are nan or 0 

    # just look at self-contradictory rows
    incomplete_feature_data.dropna(how = 'all') # drop rows that are all nan
    return incomplete_feature_data[incomplete_feature_data.sum(axis=1)!=0] # filter on rows that don't sum to zero

In [15]:
def replace_nans(df_train, df_test, label_columns, numeric_columns):
    
    df_train[label_columns] = df_train[label_columns].fillna('NA')
    df_train[numeric_columns] = df_train[numeric_columns].fillna(0)

    df_test[label_columns] = df_test[label_columns].fillna('NA')
    df_test[numeric_columns] = df_test[numeric_columns].fillna(0)
    
    return df_train, df_test    

#### Basement variables

In [16]:
# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2 all have legitimate NA values
# BsmtHalfBath, BsmtFullBath, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF are related numerical columns
#    that have missing values
# these can all be checked against each other

In [17]:
label_bsmt_columns = ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
numeric_bsmt_columns = ['Bsmt Half Bath', 'Bsmt Full Bath', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF']

bsmt_columns = label_bsmt_columns + numeric_bsmt_columns

get_inconsistent_rows(housing_full, bsmt_columns)

,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin Type 2,Bsmt Half Bath,Bsmt Full Bath,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF
Id,,,,,,,,,,,
445,Gd,TA,No,GLQ,NaN,0.0,1.0,1124.0,479.0,1603.0,3206.0
1797,Gd,TA,NaN,Unf,Unf,0.0,0.0,0.0,0.0,725.0,725.0
67,Gd,TA,NaN,Unf,Unf,0.0,0.0,0.0,0.0,1595.0,1595.0
2780,Gd,TA,NaN,Unf,Unf,0.0,0.0,0.0,0.0,936.0,936.0


In [18]:
# check location of data
housing_full.loc[[445,1797,67,2780],'train']

Id
445     True
1797    True
67      True
2780    True
Name: train, dtype: bool

In [19]:
# Homes with ids 1797,67,2780 assumed to have no outside exposure - impute No for Bsmt Exposure
# Home 445 has unfinished area - impute Unf for BsmtFin Type 2
# All other missing data for these columns is self-consistent so can replace nans with NA or 0

housing_train.loc[[1797,67,2780],'Bsmt Exposure'] = 'No'
housing_train.loc[445,'BsmtFin Type 2'] = 'Unf'

housing_train, housing_test = replace_nans(housing_train, housing_test, label_bsmt_columns, numeric_bsmt_columns)

#### Garage variables

In [20]:
# GarageType, GarageFinish, GarageQual, GarageCond all have legitimate NA values
# GarageArea, GarageCars, GarageYrBlt are related numerical columns that have missing values
# these can all be checked against each other

In [21]:
label_garage_columns = ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']
numeric_garage_columns = ['Garage Area', 'Garage Cars', 'Garage Yr Blt']

garage_columns = label_garage_columns + numeric_garage_columns

get_inconsistent_rows(housing_full, garage_columns)

,Garage Type,Garage Finish,Garage Qual,Garage Cond,Garage Area,Garage Cars,Garage Yr Blt
Id,,,,,,,
1357,Detchd,NaN,NaN,NaN,360.0,1.0,NaN


In [22]:
# check location of data
housing_full.loc[[1357],'train']

Id
1357    False
Name: train, dtype: bool

In [23]:
# Fill NaNs in this row with reasonable values - most common for finish/qual/condition
# Take common values from training set so not to build model on test data

housing_test.loc[1357,'Garage Finish'] = housing_train['Garage Finish'].value_counts().index[0]
housing_test.loc[1357,'Garage Qual'] = housing_train['Garage Qual'].value_counts().index[0]
housing_test.loc[1357,'Garage Cond'] = housing_train['Garage Cond'].value_counts().index[0]

In [24]:
# Fill garage yr blt with year house built, for all houses with no garage
housing_train.loc[housing_train['Garage Yr Blt'].isnull(),'Garage Yr Blt'] = housing_train['Year Built']
housing_test.loc[housing_test['Garage Yr Blt'].isnull(),'Garage Yr Blt'] = housing_test['Year Built']

In [25]:
# All other missing data for these columns is self-consistent so can replace nans with NA or 0
housing_train, housing_test = replace_nans(housing_train, housing_test, label_garage_columns, numeric_garage_columns)

#### Pool variables

In [26]:
# Pool QC can be checked against PoolArea
get_inconsistent_rows(housing_full, ['Pool QC','Pool Area'])

,Pool QC,Pool Area
Id,,


In [27]:
# All missing data for these columns is self-consistent so can replace nans with NA or 0
housing_train, housing_test = replace_nans(housing_train, housing_test, ['Pool QC'], ['Pool Area'])

#### Fireplace variables

In [28]:
# Fireplace Qu can be checked against Fireplaces
get_inconsistent_rows(housing_full, ['Fireplace Qu','Fireplaces'])

,Fireplace Qu,Fireplaces
Id,,


In [29]:
# All missing data for these columns is self-consistent so can replace nans with NA or 0
housing_train, housing_test = replace_nans(housing_train, housing_test,  ['Fireplace Qu'],['Fireplaces'])

#### Misc Feature variables

In [30]:
# Misc Feature can be checked against Misc Val
get_inconsistent_rows(housing_full, ['Misc Feature','Misc Val'])

,Misc Feature,Misc Val
Id,,


In [31]:
# All missing data for these columns is self-consistent so can replace nans with NA or 0
housing_train, housing_test = replace_nans(housing_train, housing_test, ['Misc Feature'],['Misc Val'])

#### MasVnr variables

In [32]:
# MasVnrType can be checked against MasVnrArea
get_inconsistent_rows(housing_full, ['Mas Vnr Type','Mas Vnr Area'])

,Mas Vnr Type,Mas Vnr Area
Id,,


In [33]:
# All missing data for these columns is self-consistent so can replace nans with NA or 0
housing_train, housing_test = replace_nans(housing_train, housing_test, ['Mas Vnr Type'],['Mas Vnr Area'])

#### Alley and Fence

In [34]:
# no values to cross-reference against, so assume all should be 'NA'
housing_train, housing_test = replace_nans(housing_train, housing_test, ['Alley'],[])
housing_train, housing_test = replace_nans(housing_train, housing_test, ['Fence'],[])

### Other columns with missing data

#### Lot Frontage

In [35]:
# look at Lot Frontage
print(housing_full[housing_full['Lot Frontage'].isnull()]['Lot Config'].unique())
housing_full[housing_full['Lot Frontage']==0]

['CulDSac' 'Inside' 'Corner' 'FR2' 'FR3']


,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,train
Id,,,,,,,,,,,,,,,,,,,,,


In [36]:
# no houses have a zero lot frontage
# these are therefore missing entries, rather than zero entries - fill with train average for that lot config
lt_mean = housing_train.groupby('Lot Config')['Lot Frontage'].mean()

housing_train.loc[housing_train['Lot Frontage'].isnull(), 'Lot Frontage'] = [lt_mean[config] for config in housing_train.loc[housing_train['Lot Frontage'].isnull(), 'Lot Config']]
housing_test.loc[housing_test['Lot Frontage'].isnull(), 'Lot Frontage'] = [lt_mean[config] for config in housing_test.loc[housing_test['Lot Frontage'].isnull(), 'Lot Config']]

#### Electrical

In [37]:
# no other relevant columns to look at
# fill with most common value from training data
electrical_mode = housing_train['Electrical'].value_counts().index[0]
housing_train.loc[housing_train['Electrical'].isnull(), 'Electrical'] = electrical_mode
housing_test.loc[housing_test['Electrical'].isnull(), 'Electrical'] = electrical_mode

### Final check of completeness

In [38]:
print('Number of null values in training set:', housing_train.isnull().sum().sum())
print('Number of null values in test set:', housing_test.isnull().sum().sum())

Number of null values in training set: 0
Number of null values in test set: 0


## 3. Data types

### Categorical to numeric

In [39]:
# create maps to transform categories that should be ordinal

# Street
map_street = {'Grvl':0, 'Pave':1}

# Alley
map_alley = {'NA':0, 'Grvl':1, 'Pave':2}

# Lot shape
map_shape = {'Reg':0, 'IR1':1, 'IR2':2, 'IR3':3}

# Land contour
map_contour = {'Lvl':3, 'Bnk':2, 'HLS':1, 'Low':0}

# Utilities
map_util = {'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1}

# Land slope
map_slope = {'Gtl':1, 'Mod':2, 'Sev':3}

# ExterQual, ExterCond, BsmtQual, BsmtCond, HeatingQC, KitchenQual, 
# FireplaceQu, GarageQual, GarageCond, PoolQc
map_qual = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}

# BsmtExposure
map_exp = {'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0}

# BsmtFinType1, 2
map_btype = {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0}

# Central air
map_air = {'Y':1,'N':0}

# Functional
map_func = {'Typ':0, 'Min1':1, 'Min2':2, 'Mod':3, 'Maj1':4, 'Maj2':5, 'Sev':6, 'Sal':7}

# GarageFinish
map_garagefin = {'Fin':3,'RFn':2, 'Unf':1, 'NA':0}

# PavedDrive
map_drive = {'Y':2,'P':1,'N':0}

In [40]:
def transform_cat_to_ord(df):
    
    """ transforms categories that should be ordinal in nature to numeric values """
    
    df.loc[:,'Street'] = df['Street'].map(map_street)
    df.loc[:,'Alley'] = df['Alley'].map(map_alley)
    df.loc[:,'Lot Shape'] = df['Lot Shape'].map(map_shape)
    df.loc[:,'Land Contour'] = df['Land Contour'].map(map_contour)
    df.loc[:,'Utilities'] = df['Utilities'].map(map_util)
    df.loc[:,'Land Slope'] = df['Land Slope'].map(map_slope)
    df.loc[:,'Exter Qual'] = df['Exter Qual'].map(map_qual)
    df.loc[:,'Exter Cond'] = df['Exter Cond'].map(map_qual)
    df.loc[:,'Bsmt Qual'] = df['Bsmt Qual'].map(map_qual)
    df.loc[:,'Bsmt Cond'] = df['Bsmt Cond'].map(map_qual)
    df.loc[:,'Heating QC'] = df['Heating QC'].map(map_qual)
    df.loc[:,'Kitchen Qual'] = df['Kitchen Qual'].map(map_qual)
    df.loc[:,'Fireplace Qu'] = df['Fireplace Qu'].map(map_qual)
    df.loc[:,'Garage Qual'] = df['Garage Qual'].map(map_qual)
    df.loc[:,'Garage Cond'] = df['Garage Cond'].map(map_qual)
    df.loc[:,'Pool QC'] = df['Pool QC'].map(map_qual)
    df.loc[:,'Bsmt Exposure'] = df['Bsmt Exposure'].map(map_exp)
    df.loc[:,'BsmtFin Type 1'] = df['BsmtFin Type 1'].map(map_btype)
    df.loc[:,'BsmtFin Type 2'] = df['BsmtFin Type 2'].map(map_btype)
    df.loc[:,'Central Air'] = df['Central Air'].map(map_air)
    df.loc[:,'Functional'] = df['Functional'].map(map_func)
    df.loc[:,'Garage Finish'] = df['Garage Finish'].map(map_garagefin)
    df.loc[:,'Paved Drive'] = df['Paved Drive'].map(map_drive)
    
    return df

In [41]:
housing_train = transform_cat_to_ord(housing_train)
housing_test = transform_cat_to_ord(housing_test)

### Numeric to categorical

In [42]:
## MSSubclass is saved as a numerical value, but is really a category

In [43]:
map_subclass = {20:'A', 30:'B', 40:'C', 45:'D', 50:'E', 60:'F', 70:'G', 75:'H', 
                80:'I', 85:'J', 90:'K', 120:'L', 150:'M', 160:'N', 180:'O', 190:'P'}

In [44]:
def transform_num_to_cat(df):
    
    """ transforms numerics that should be categorical in nature to strings """
    
    df.loc[:,'MS SubClass'] = df['MS SubClass'].map(map_subclass)
    
    return df

In [45]:
housing_train = transform_num_to_cat(housing_train)
housing_test = transform_num_to_cat(housing_test)

## 4. Save clean data

In [46]:
housing_train.to_csv('../data/processed/train_clean.csv')
housing_test.to_csv('../data/processed/test_clean.csv')